In [39]:
from langchain_core.output_parsers import PydanticOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_ollama import ChatOllama
from typing import List, Optional
from langchain_core.pydantic_v1 import BaseModel, Field

# Model

In [40]:
class Person(BaseModel):
    """Information about a person."""

    name: Optional[str] = Field(default=None, description="The name of the person.")
    birth_date: Optional[str] = Field(default=None, description="The person's birth date")
    birth_place: Optional[str] = Field(default=None, description="The person's birth place")
    death_date: Optional[str] = Field(default=None, description="The person's death date")
    death_place: Optional[str] = Field(default=None, description="The person's death place")
    religion: Optional[str] = Field(default=None, description="The person's religion")
    origins: Optional[str] = Field(default=None, description="The person's origins")
    father_name: Optional[str] = Field(default=None, description="The person's father name")
    father_job: Optional[str] = Field(default=None, description="The person's father job (or title, or position, ...)")
    mother_name: Optional[str] = Field(default=None, description="The person's mother name")
    mother_job: Optional[str] = Field(default=None, description="The person's mother job (or title, or position, ...)")
    partner_name: Optional[str] = Field(default=None, description="The person's partner's name (eg husband or wife)")
    partner_nationality: Optional[str] = Field(default=None, description="The person's partner's nationality")
    partner_parents: Optional[List[str]] = Field(default=None, description="The person's partner's parent names")
    formation: Optional[str] = Field(default=None, description="The person's formation")
    formation_place: Optional[str] = Field(default=None, description="The person's formation location")
    job: Optional[List[str]] = Field(default=None, description="The person's job (or place, responsabilities, ...)")


class Model(BaseModel):
    
    persons: Optional[List[Person]]

# LangChain

In [41]:
# Define the parser
parser = PydanticOutputParser(pydantic_object=Model)

# Define prompt
prompt = ChatPromptTemplate.from_messages([
        ("system", "Answer the user query. Wrap the output in ```json and ``` tags\n{format_instructions}. Do not provide more fields that the one required."),
        ("human", "{text}")
    ]).partial(format_instructions=parser.get_format_instructions())

# Define the LLM
llm = ChatOllama(model="llama3.1", temperature=0)

# Build the chain
chain = prompt | llm | parser


## Extract information

In [ ]:
initial_text = """
René Dubois naît le 17.8.1905 aux Verrières, et meurt le 16.11.1976 à Marin-Epagnier, protestant, de Buttes.
Fils d'Henri Adolphe, fonctionnaire postal.
Marié(e) à Suzanne Mina Arnoux, Française, fille de Jules Cyprien.
Licence en sciences économiques à Neuchâtel.
Officier instructeur dès 1929.
Chef d'état-major des troupes d'aviation et de DCA (1953-1955).
Commandant de la brigade légère 1, de la division 2 (1958-1961), du corps d'armée de campagne 1 (1962-1967).
"""

model = chain.invoke({"text": initial_text})

for person in model.persons:
    for key, value in person.dict().items():
        print(key, '-', getattr(person, key))

In [ ]:
def analyze_chain(text):

    print('=== INITIAL TEXT ===')
    print(text)

    prompt_result = prompt.invoke({"text": initial_text})
    print('=== PROMPT ===')
    for message in prompt_result.messages:
        print(message.content)
        print('---')

    llm_result = llm.invoke(prompt_result)
    print('=== LLM ANSWER ===')
    print(llm_result.content)

    parser_result = parser.invoke(llm_result)
    print('=== PARSED ANSWER ===')
    print(parser_result)


analyze_chain(initial_text)

___

In [ ]:
from typing import Optional, List
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.pydantic_v1 import BaseModel, Field
from langchain_ollama import ChatOllama
from langchain_core.pydantic_v1 import BaseModel, Field


class Person(BaseModel):
    """Information about a person."""
    name: Optional[str] = Field(default=None, description="The name")
    gender: Optional[str] = Field(default=None, description="Gender (male or female)")
    origins: Optional[str] = Field(default=None, description="The origin (a geographical name)")
    religion: Optional[str] = Field(default=None, description="The religion")
    # birth_place: Optional[str] = Field(default=None, description="the person birth date (format: yyyy.mm.dd)")
    # birth_date: Optional[str] = Field(default=None, description="the person geographical birth place")
    # death_place: Optional[str] = Field(default=None, description="the person death date (format: yyyy.mm.dd)")
    # death_date: Optional[str] = Field(default=None, description="the person geographical death place")
    father_name: Optional[str] = Field(default=[], description="the father name")
    mother_name: Optional[str] = Field(default=[], description="the mother name")
    


llm = ChatOllama(model="llama3.1", temperature=0, base_url="http://127.0.0.1:5000")
prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "As an expert extraction algorithm, extract relevant information about Suzanne Mina Arnoux."
        ),
        ("human", "{text}"),
    ]
)

runnable = prompt | llm.with_structured_output(schema=Person)

text = """
René Dubois naît le 17.8.1905 aux Verrières, et meurt le 16.11.1976 à Marin-Epagnier, protestant, de Buttes.
Fils d'Henri Adolphe, fonctionnaire postal.
Marié(e) à Suzanne Mina Arnoux, Française, fille de Jules Cyprien.
Licence en sciences économiques à Neuchâtel.
Officier instructeur dès 1929.
Chef d'état-major des troupes d'aviation et de DCA (1953-1955).
Commandant de la brigade légère 1, de la division 2 (1958-1961), du corps d'armée de campagne 1 (1962-1967).
"""
runnable.invoke({"text": text.strip()})

In [ ]:
# Way 1
# With tools
# Does not seems to work so well


from typing import Optional
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.pydantic_v1 import BaseModel, Field
from langchain_ollama import ChatOllama
from langchain_core.pydantic_v1 import BaseModel, Field


class Person(BaseModel):
    """Information about a person."""

    name: Optional[str] = Field(default=None, description="The name of the person")
    gender: Optional[str] = Field(default=None, description="The gender of the person ('m' or 'f')")
    father: Optional[str] = Field(default=None, description="The father name of the person")
    partner: Optional[str] = Field(default=None, description="The partner (husband or spouse) of the person")


prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are an expert extraction algorithm. "
            "Only extract relevant information from the text. "
            "If you do not know the value of an attribute asked to extract, "
            "return null for the attribute's value."
            "Your task is to focus on information about Suzanne Mina Arnoux: the person name is then Suzanne Mina Arnoux."
        ),
        ("human", "{text}"),
    ]
)


llm = ChatOllama(model="llama3.1", temperature=0)

runnable = prompt | llm.with_structured_output(schema=Person)

text = """
René Dubois naît le 17.8.1905 aux Verrières, et meurt le 16.11.1976 à Marin-Epagnier, protestant, de Buttes.
Fils d'Henri Adolphe, fonctionnaire postal.
Marié(e) à Suzanne Mina Arnoux, Française, fille de Jules Cyprien.
Licence en sciences économiques à Neuchâtel.
Officier instructeur dès 1929.
Chef d'état-major des troupes d'aviation et de DCA (1953-1955).
Commandant de la brigade légère 1, de la division 2 (1958-1961), du corps d'armée de campagne 1 (1962-1967).
"""
runnable.invoke({"text": text.strip()})

In [ ]:
# Way 2, the "old" way

from typing import List, Optional
from langchain_core.output_parsers import PydanticOutputParser
from langchain_core.pydantic_v1 import BaseModel, Field
from langchain_ollama import ChatOllama
from langchain_core.prompts import ChatPromptTemplate


class Person(BaseModel):
    """Information about a person."""

    name: Optional[str] = Field(default=None, description="The name of the person")
    gender: Optional[str] = Field(default=None, description="The gender of the person ('m' or 'f')")
    father: Optional[str] = Field(default=None, description="The father name of the person")
    partner: Optional[str] = Field(default=None, description="The partner (husband or spouse) of the person")


# Define the parser
parser = PydanticOutputParser(pydantic_object=Person)


# Define prompt
prompt = ChatPromptTemplate.from_messages([
        ("system", 
            "Answer the user query. "
            "Wrap the output in ```json and ``` tags\n{format_instructions}. "
            "Do not provide more fields that the one required. "
        ),
        ("human", "{text}")
    ]).partial(format_instructions=parser.get_format_instructions())


# Define the LLM
llm = ChatOllama(model="llama3.1", temperature=0)

# Build the chain
chain = prompt | llm | parser

text = """
From the following text, what do we learn about Henri Adolphe (only!):

René Dubois naît le 17.8.1905 aux Verrières, et meurt le 16.11.1976 à Marin-Epagnier, protestant, de Buttes.
Fils d'Henri Adolphe, fonctionnaire postal.
Marié(e) à Suzanne Mina Arnoux, Française, fille de Jules Cyprien.
Licence en sciences économiques à Neuchâtel.
Officier instructeur dès 1929.
Chef d'état-major des troupes d'aviation et de DCA (1953-1955).
Commandant de la brigade légère 1, de la division 2 (1958-1961), du corps d'armée de campagne 1 (1962-1967).
"""
chain.invoke({"text": text.strip()})


In [1]:
import gmpykit as kit

prompt = """
SYSTEM: You are an expert and strict verificator. From the context, answer the user query using 'True' or 'False'. Your answer should be a single word.
Context:
René Dubois naît le 17.8.1905 aux Verrières, et meurt le 16.11.1976 à Marin-Epagnier, protestant, de Buttes.
Fils d'Henri Adolphe, fonctionnaire postal.
Marié(e) à Suzanne Mina Arnoux, Française, fille de Jules Cyprien.
Licence en sciences économiques à Neuchâtel.
Officier instructeur dès 1929.
Chef d'état-major des troupes d'aviation et de DCA (1953-1955).
Commandant de la brigade légère 1, de la division 2 (1958-1961), du corps d'armée de campagne 1 (1962-1967).
USER: Is the assertion 'René Dubois was a Fonctionnaire postal.' true? If it is not clearly known or said, the answer should be 'False'
"""


print(kit.wrap(kit.ask_llm(prompt), 80))

True


In [2]:
import knex